<a href="https://colab.research.google.com/github/snowy-the-arctic-fox/Sounds-of-Fractals/blob/main/Sounds_of_Fractals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sounds of Fractals
In the [previous code](https://github.com/snowy-the-arctic-fox/Fractal-Generator/tree/main), generated a fractal apon a few values/inputs from the user. These inputs consists of `Julian Constant 1`, `Julian Constant 2`, `Division Factor`. in this code, the fractal is turned into an audio file! A spectrogram was generated to display the frequency content of an audio signal over time. The `plt.specgram()` function from the matplotlib library was used to calculate the spectrogram, which represents the magnitude of the Fourier transform of overlapping short-time segments of the audio signal. The frequency content of each segment is displayed as a vertical bar for each frequency bin, while time progresses horizontally. 

The logarithmic scale for frequencies was set up by applying the `np.log10()` function to the frequency bins, which helps to better visualize the distribution of frequency components. A colormap was used to assign a different color to each frequency bin, with the color intensity representing the magnitude of each frequency component. 

However, a warning message appeared indicating that there were "invalid value encountered in log10". This may be due to the presence of zero or negative values in the frequency bins, which cause the logarithm function to return an undefined value. To avoid this issue, a small constant value was added to the frequency bins before applying the `np.log10()` function.

In [ ]:
#@title Install { vertical-output: true }
!pip install numpy matplotlib librosa pydsm

In [ ]:
#@title Initial Julia Definition
import numpy as np
import matplotlib.pyplot as plt

def generate_julia(width, height, max_iter, c):
    x = np.linspace(-2.5, 1.5, width)
    y = np.linspace(-2, 2, height)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y

    fractal = np.zeros(Z.shape, dtype=int)
    z = Z.copy()
    
    for i in range(max_iter):
        mask = np.abs(z) < 2
        z[mask] = z[mask] ** 2 + c
        fractal += mask
    
    return fractal

# Set the dimensions of the fractal image
width = 640
height = 480

In [ ]:
#@title Fractal Generator
# Set the maximum number of iterations
max_iter = int(input("Max iterations: "))

# Get the user inputs for the Julia set numbers
j1 = float(input("Julia constant factor 1: "))
j2 = float(input("Julia constant factor 2: "))
div = float(input("Division Factor: "))

# Set the constant 'c' for the Julia set
c = (j1 + j2) / div

# Generate the fractal
fractal = generate_julia(width, height, max_iter, c)


In [ ]:
#@title Fractal Out
# Display the fractal image
plt.figure(figsize=(8, 6))
plt.imshow(fractal, cmap='hot', extent=(-2.5, 1.5, -2, 2))
plt.title('Julia Set Fractal')
plt.xlabel('Re(c)')
plt.ylabel('Im(c)')
plt.colorbar(label='Iterations')
plt.show()

In [ ]:
#@title Generate Audio
# Generate the audio signal
import numpy as np

# Set the audio parameters
duration = 5  # Duration of the audio in seconds
sample_rate = 44100  # Sample rate (number of samples per second)

# Generate a random audio signal
t = np.linspace(0, duration, duration * sample_rate, endpoint=False)
audio = np.random.uniform(-1, 1, len(t))

# Normalize the audio signal
normalized_audio = audio / np.max(np.abs(audio))
scaling_factor = 0.9 * (np.iinfo(np.int16).max)
audio = (normalized_audio * scaling_factor).astype(np.int16)

# Save the audio signal as a WAV file
from scipy.io import wavfile

output_file = 'fractal_audio.wav'

# Normalize the audio signal
normalized_audio = audio / np.max(np.abs(audio))
scaling_factor = 0.9 * (np.iinfo(np.int16).max)
audio = (normalized_audio * scaling_factor).astype(np.int16)

# Save the audio signal as a WAV file
wavfile.write(output_file, sample_rate, audio)


In [ ]:
#@title Download and Load Audio
from IPython.display import Audio
from google.colab import files

# Download the audio file
files.download('fractal_audio.wav')

audio_file = 'fractal_audio.wav'

# Display the audio widget
Audio(audio_file)

In [ ]:
#@title Generate Waterfall Visualization (log)
import numpy as np
import matplotlib.pyplot as plt

# Calculate the spectrogram of the audio signal
_, _, _, spectrogram = plt.specgram(audio, Fs=sample_rate)

# Get the spectrogram data from the AxesImage object
spectrogram_data = spectrogram.get_array()

# Get the number of frequency bins and time steps from the spectrogram data
num_freq_bins = spectrogram_data.shape[0]
num_time_steps = spectrogram_data.shape[1]

# Set up the logarithmic scale for frequencies
freqs = np.fft.fftfreq(num_freq_bins, d=1/sample_rate)[1:]
log_frequencies = np.log10(np.where(freqs > 0, freqs, 1e-9))

# Set up the colormap
cmap = plt.get_cmap('jet')

# Display the waterfall plot of the audio signal with color-coded frequencies
plt.figure(figsize=(4, 3))
plt.imshow(log_frequencies.reshape(-1, 1), extent=[0, num_time_steps, 0, 1], aspect='auto', cmap=cmap, origin='lower')